# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import sklearn

from azureml.core import Dataset, Environment, Experiment, Workspace
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

## Dataset

### Overview
In this experiment we will use the Azure Machine Learning Services and AutoML functionality to predict Heart failure Mortaility.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

In [3]:
dataset = Dataset.get_by_name(ws, name='heartfailure')
heart_failure = dataset.to_pandas_dataframe()

## Create Experiment

In [4]:
# choose a name for experiment
experiment_name = 'automl_heartfailure'
experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl_heartfailure,davidudacitycapstone,Link to Azure Machine Learning studio,Link to Documentation


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below

In [5]:

# TODO: Put your automl config here
automl_settings = {
    "task": 'classification',
    "iterations": 30,
    "experiment_timeout_minutes": 15,
    "primary_metric": 'accuracy',
    "training_data": dataset,
    "label_column_name": 'DEATH_EVENT',
    "n_cross_validations": 3,
    "max_concurrent_iterations": 8,
    "max_cores_per_iteration": -1
}


automl_config = AutoMLConfig(**automl_settings)

## Run Details

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
from azureml.widgets import RunDetails

remote_run = experiment.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=False)

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_3bca6b22-132a-4b79-8deb-fadb9dcd3e3d',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-11-22T21:02:10.875245Z',
 'endTimeUtc': '2020-11-22T21:19:44.819104Z',
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_heartfailure","subscription_id":"641a5604-2a7e-401d-a98d-ee5182f46af8","resource_group":"djs-test-e","workspace_name":"davidudacitycapstone","region":"eastus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":30,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classe

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
best_automl_run, best_model = remote_run.get_output()
print(best_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('22',
                                           Pipeline(memory=None,
                                                    steps=[('minmaxscaler',
                                                            MinMaxScaler(copy=True,
                                                                         feature_range=(0,
                                                                                        1))),
                                                           ('randomforestclassifier',
                                                            RandomForestClassifier(bootstrap=True,
                                                                                   ccp_alpha=0.0,
                                                                                   class_weight=None,
                                                                                   criterion='entr

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
model = best_automl_run.register_model(model_name = "best_heart_failure.pkl", model_path = './outputs/')
model.download() 

'outputs'

In [37]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=2)
environment = Environment.from_conda_specification('udacitymlhf', './outputs/conda_env_v_1_0_0.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=environment)

service = model.deploy(
    workspace=ws,
    name="udacitymlhf",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [38]:
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [34]:
rid = 8
x = heart_failure.iloc[rid, :-1].to_dict()
y = heart_failure.iloc[rid, -1]

In [35]:
import json
import numpy as np
input_payload = json.dumps({
    'data': x
})

output = service.run(input_payload)
print(f"Predicted: {output}\nExpected: {y}")

Predicted: [1]
Expected: 1


TODO: In the cell below, print the logs of the web service and delete the service

In [39]:
logs = service.get_logs()
print(logs)
service.delete()

2020-11-22T23:00:41,653956656+00:00 - iot-server/run 
2020-11-22T23:00:41,655338280+00:00 - rsyslog/run 
2020-11-22T23:00:41,655870089+00:00 - gunicorn/run 
2020-11-22T23:00:41,659974658+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_99474c16f35a92149d1cc6dec149d8e9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_99474c16f35a92149d1cc6dec149d8e9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_99474c16f35a92149d1cc6dec149d8e9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_99474c16f35a92149d1cc6dec149d8e9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_99474c16f35a92149d1cc6dec149d8e9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd